*Das [LGLN] stellt zahlreiche offene Geodaten bereit. Diese Jupyter-Notebook-Serie demonstriert den Zugriff mit Python.*
# Sentinel-2-Daten
**Die Daten werden täglich aktualisiert. Wenn neue, wolkenfreie Sentinel-2-Aufnahmen entstanden sind, stehen diese ab ca. 23:30 Uhr zur Verfügung.** 
 - Die [Sentinel-2]-Daten sind Teil des [Copernicus]-Programms der Europäischen Union
 - Verarbeitung und Bereitstellung durch das [LGLN] auf [CODE-DE]
 - Feedback zu den Daten an: jonas.bostelmann@lgln.niedersachsen.de
 - Lizenz: [Creative Commons (CC BY 4.0)], Enthält modifzierte Copernicus Sentinel Daten 2021, verarbeitet durch das Landesamt für Geoinformation und Landesvermessung Niedersachsen (LGLN)
 
[Sentinel-2]: https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi
[Copernicus]: https://www.copernicus.eu/de
[CODE-DE]: https://code-de.org/de/
[LGLN]: www.lgln.niedersachsen.de
[Creative Commons (CC BY 4.0)]: https://creativecommons.org/licenses/by/4.0/deed.de

### Installation benötigter Python Pakete
https://github.com/BostelmannLGLN/LGLN-OpenData-Notebooks/blob/main/requirements.txt

In [ ]:
# !pip install -r requirements.txt

#### Imports

In [ ]:
import rasterio
from rasterio.plot import show, plotting_extent
import matplotlib.pyplot as plt
import numpy as np
import functions

## Sentinel-2-Kacheln - 8 km x 8 km - Wenig Wolken - True Color Image (TCI) - 10 m Bodenpixelgröße

### Download einer einzelnen Kachel

In [ ]:
endpoint_url = 'https://cloud.code-de.org:8080/swift/v1/AUTH_622a231224cb46c6982643e55e817c98/'
bucket_name = 'tiles'

be8_nr = '8-25405800'
datensatz = 'S2'
datum = '20200921'
dateityp = 'TCI_10m.tif'

obj = f'{be8_nr}/{datensatz}/{datum}/{be8_nr}_{datensatz}_{datum}_TCI_10m.tif'

url = endpoint_url + f'{bucket_name}/{obj}'

print(f'Download-URL: \n{url}')

### Laden und Anzeigen mit Rasterio

In [ ]:
with rasterio.open(url) as image:
        plt.figure(figsize=(10, 10))    
        ax = show(image.read())

## Anzahl verfügbarer Kacheln anzeigen

### Gebiet auswählen

In [ ]:
landkreis = 'Hannover' # None for all of Niedersachsen

name, be_name, area, tiles = functions.load_tiles(region='NI', landkreis=landkreis, use_cos=True)

ax = tiles.plot(figsize=(15, 15), legend=True, edgecolor='black')
area.boundary.plot(ax=ax, edgecolor='black')

tiles

### Zeitraum wählen und Daten laden

In [ ]:
date_str = '2021'

bucket = {
    'endpoint_url': endpoint_url,
    'name': bucket_name
}

tiles, dates, tiles_count = functions.available_dates(tiles, date_str, bucket)
tiles

### Plotten

In [ ]:
ax = tiles.plot(column='S2', figsize=(15, 15), legend=True, edgecolor='black')
area.boundary.plot(ax=ax, edgecolor='black')

tiles.apply(lambda x: ax.annotate(text=int(x.S2), xy=x.geometry.centroid.coords[0], ha='center', color='red', fontsize='x-large'), axis=1);

## Sentinel-2-Landkreis-Mosaike - Wenig Wolken

In [ ]:
endpoint_url = 'https://cloud.code-de.org:8080/swift/v1/AUTH_622a231224cb46c6982643e55e817c98/'
bucket_name = 'mosaics'

landkreis = landkreis
datum = '20200921'
datensatz = 'S2'

mosaic_obj = f'{landkreis}_{datum}_{datensatz}.tif'

mosaic_url = endpoint_url + f'{bucket_name}/{mosaic_obj}'

print(f'Download-URL: \n{mosaic_url}')

In [ ]:
with rasterio.open(mosaic_url) as mosaic:
    mosaic_data = mosaic.read()
    mosaic_data[mosaic_data==0] = np.max(mosaic_data) # weißer hintergrund
    mosaic_extend = plotting_extent(mosaic)
    print(mosaic.profile)
    plt.figure(figsize=(15, 15))    
    ax = show(mosaic_data)

In [ ]:
name, be_name, area, tiles = functions.load_tiles(region='NI', use_cos=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(mosaic_data, ax=ax, extent=mosaic_extend)
area.plot(ax=ax, legend=True, edgecolor='red', facecolor='none')
area.apply(lambda x: ax.annotate(text=x.BEZ_KRS, 
                                 xy=x.geometry.centroid.coords[0], 
                                 ha='center', color='black', 
                                 fontsize='medium'), 
                                 axis=1);